In [1]:
import numpy as np
from seglearn.base import TS_Data
from seglearn.pipe import Pype
import numpy as np
from seglearn.base import TS_Data
from seglearn.pipe import Pype
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import time

import os
os.chdir(r"E:\MS_AI\Sem-IV\Seminar-ADLTS\tool-tracking\data-tools")
# os.getcwd()
from datatools import to_ts_data
from datatools import MeasurementDataReader, Measurement, MeasurementSeries, Action, Tool, Config, MeasurementSeries, DataTypes

os.chdir(r"E:\MS_AI\Sem-IV\Seminar-ADLTS\tool-tracking\fhg-utils")
from fhgutils import Segment, contextual_recarray_dtype, filter_ts_data
os.chdir(r"E:\MS_AI\Sem-IV\Seminar-ADLTS\tool-tracking")

from fhgutils import Segment, contextual_recarray_dtype, filter_ts_data
import pandas as pd


In [80]:
# put data into this source folder
source = "./tool-tracking-data/"

mdr = MeasurementDataReader(source=source)
q = mdr.query(query_type=Measurement)
# mytool = "electric_screwdriver"
mytool = "pneumatic_rivet_gun"
# mytool = "pneumatic_screwdriver"
#mytool = "test_tool"
filtered_query = q.filter_by(Tool == mytool)
data_dict = filtered_query.get()

# mytool = "electric_screwdriver"
'''{14: 'tightening_double',
 7: 'shaking',
 8: 'undefined',
 2: 'tightening',
 5: 'motor_activity_ccw',
 4: 'motor_activity_cw',
 3: 'untightening',
 6: 'manual_motor_rotation'}'''

# "pneumatic_screwdriver"
'''
{7: 'shaking',
 5: 'motor_activity_ccw',
 3: 'untightening',
 38: 'impact',
 8: 'undefined',
 2: 'tightening',
 4: 'motor_activity_cw',
 14: 'tightening_double'}
 '''

# "pneumatic_rivet_gun"
'''
{38: 'impact',
 30: 'pull_trigger_air_with_rivet',
 28: 'pull_trigger_not_removed',
 8: 'undefined',
 25: 'pull_trigger',
 29: 'pull_trigger_air',
 7: 'shaking'}'''

[INFO] Preparing data from:
  tool-tracking-data\pneumatic_rivet_gun\pythagoras-07-20200724
  tool-tracking-data\pneumatic_rivet_gun\pythagoras-07-20200724
  tool-tracking-data\pneumatic_rivet_gun\pythagoras-07-20200724
  tool-tracking-data\pneumatic_rivet_gun\pythagoras-07-20200724


[INFO] Read annotation: 100%|██████████| 16/16 [00:07<00:00,  2.04it/s, file=data-04.annotation]


[INFO] Finished with 4 measurement(s).


"\n{38: 'impact',\n 30: 'pull_trigger_air_with_rivet',\n 28: 'pull_trigger_not_removed',\n 8: 'undefined',\n 25: 'pull_trigger',\n 29: 'pull_trigger_air',\n 7: 'shaking'}"

In [108]:
Xt, Xc, y = to_ts_data(data_dict, contextual_recarray_dtype)
# Converts raw DataBunch into:
# Xt: list of time series (NumPy arrays)
# Xc: contextual metadata (recarray with info like ID, desc, sr)
# y: labels for each time series (same length as Xt)


In [84]:
X = TS_Data(Xt, Xc)

## for Rivet Gun skip mic data before processing

In [110]:
mask = Xc.desc != 'mic'

Xt_filtered = [x for x, keep in zip(Xt, mask) if keep]
Xc_filtered = Xc[mask]
y_filtered = [label for label, keep in zip(y, mask) if keep]

X_filtered = TS_Data(Xt_filtered, Xc_filtered)

In [ ]:
pipe = Pype([
    ('segment', Segment(window_length=0.4, overlap=0.5, enforce_size=True, n=len(np.unique(Xc_filtered.desc))))
])

# pipe = Pype([
#     ('segment', Segment(window_length=0.4, overlap=0.5, enforce_size=True, n=len(np.unique(Xc.desc))))
# ])

In [ ]:
X_trans, y_trans = pipe.fit_transform(X_filtered, y_filtered)
# X_trans, y_trans = pipe.fit_transform(X, y)

[INFO] segment ['acc' 'gyr' 'mag'] together
[INFO] segment ['acc' 'gyr' 'mag'] together
[INFO] segment ['acc' 'gyr' 'mag'] together
[INFO] segment ['acc' 'gyr' 'mag'] together


Segment: 100%|██████████| 32388/32388 [00:01<00:00, 25244.90it/s]


In [115]:
print("[INFO] extract segmented ACC data")
Xt_acc, Xc_acc, y_acc = filter_ts_data(X_trans, y_trans, filt={'desc': ['acc']})
print('Sampling Rate: ',Xc_acc[0].sr)
print("[INFO] extract segmented GYR data")
Xt_gyr, Xc_gyr, y_gyr = filter_ts_data(X_trans, y_trans, filt={'desc': ['gyr']})
print('Sampling Rate: ',Xc_gyr[0].sr)
print("[INFO] extract segmented MAG data")
Xt_mag, Xc_mag, y_mag = filter_ts_data(X_trans, y_trans, filt={'desc': ['mag']})
print('Sampling Rate: ',Xc_mag[0].sr)
# print("[INFO] extract segmented MIC data")
# Xt_mic, Xc_mic, y_mic = filter_ts_data(X_trans, y_trans, filt={'desc': ['mic']})
# print('Sampling Rate: ',Xc_mic[0].sr)

[INFO] extract segmented ACC data
Sampling Rate:  102.29
[INFO] extract segmented GYR data
Sampling Rate:  102.29
[INFO] extract segmented MAG data
Sampling Rate:  154.966


In [116]:
print("[INFO] shape of ACC data is", Xt_acc.shape)
print(f"[INFO] {Xt_acc.shape[0]} windows with {Xt_acc[0].shape[1]} sample each (time stamps and 3 axis accelerometer)")

print("[INFO] shape of GYR data is", Xt_gyr.shape)
print(f"[INFO] {Xt_gyr.shape[0]} windows with {Xt_gyr[0].shape[1]} sample each (time stamps and 3 axis Gyr)")

print("[INFO] shape of MAG data is", Xt_mag.shape)
print(f"[INFO] {Xt_mag.shape[0]} windows with {Xt_mag[0].shape[1]} sample each (time stamps and 3 axis Mag)")

# print("[INFO] shape of MIC data is", Xt_mic.shape)
# print(f"[INFO] {Xt_mic.shape[0]} windows with {Xt_mic[0].shape[1]} sample each (time stamps and 1 axis )")

[INFO] shape of ACC data is (10796,)
[INFO] 10796 windows with 4 sample each (time stamps and 3 axis accelerometer)
[INFO] shape of GYR data is (10796,)
[INFO] 10796 windows with 4 sample each (time stamps and 3 axis Gyr)
[INFO] shape of MAG data is (10796,)
[INFO] 10796 windows with 4 sample each (time stamps and 3 axis Mag)


In [117]:
from fhgutils import filter_labels, one_label_per_window, summarize_labels

In [118]:
df_acc = pd.DataFrame({'X_acc':Xt_acc,'y_acc':y_acc})
df_gyr = pd.DataFrame({'X_gyr':Xt_gyr,'y_gyr':y_gyr})
df_mag = pd.DataFrame({'X_mag':Xt_mag,'y_mag':y_mag})
# df_mic = pd.DataFrame({'X_mic':Xt_mic,'y_mic':y_mic})

In [119]:
# df_combined = pd.concat([df_acc,df_gyr, df_mag, df_mic], axis=1)
df_combined = pd.concat([df_acc,df_gyr, df_mag], axis=1)

In [ ]:
row_of_win_with_neg1_in_one_of_the_sensor = []
for column in ['y_acc','y_gyr','y_mag','y_mic']:
    for i in range(len(df_combined)):
        if(-1 in df_combined[column][i]):
            row_of_win_with_neg1_in_one_of_the_sensor.append(i)

row_of_win_with_neg1_in_one_of_the_sensor = np.unique(row_of_win_with_neg1_in_one_of_the_sensor)

In [122]:
row_of_win_with_neg1_in_one_of_the_sensor

array([ 2754,  2755,  2756, ..., 10793, 10794, 10795])

In [123]:
df_combined = df_combined.drop(index=row_of_win_with_neg1_in_one_of_the_sensor)
df_combined = df_combined.reset_index(drop=True)

In [124]:
orig = one_label_per_window(df_combined['y_acc'])
print(len(orig),orig)
l,c = np.unique(orig,return_counts=True)
l[np.argmax(c)]

flattened 4 labels: [ 7  8 25 38]
9455 [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 25, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,

np.int64(8)

In [ ]:
columns = ['y_acc','y_gyr','y_mag','y_mic']
single_label_dict = {}

for column in columns:
    single_label_per_window = []
    for i in range(len(df_combined)):
        labels = df_combined[column][i] 
        unique_labels, counts = np.unique(labels, return_counts=True)
        final_label_for_window = int(unique_labels[np.argmax(counts)])
        single_label_per_window.append(final_label_for_window)
    single_label_dict[column] = single_label_per_window

In [126]:
df_single_label = pd.DataFrame(single_label_dict)
df_single_label

,y_acc,y_gyr,y_mag
0,8,8,8
1,8,8,8
2,8,8,8
3,8,8,8
4,8,8,8
...,...,...,...
9450,8,8,8
9451,8,8,8
9452,8,8,8
9453,8,8,8


In [ ]:
print(df_single_label['y_acc'].value_counts())
print(df_single_label['y_gyr'].value_counts())
print(df_single_label['y_mag'].value_counts())
print(df_single_label['y_mic'].value_counts())

y_acc
8     9281
7      112
38      51
25      11
Name: count, dtype: int64
y_gyr
8     9281
7      112
38      51
25      11
Name: count, dtype: int64
y_mag
8     9292
7      113
38      44
25       6
Name: count, dtype: int64


In [128]:
df_diff = {} 

In [129]:
mismatch = df_single_label['y_acc'] != df_single_label['y_gyr']
# Print mismatched rows and their indices
if mismatch.any():
    print("Rows where 'col1' and 'col2' are different:")
    df_diff['acc_gyr_diff'] = df_single_label[mismatch]
    print(df_single_label[mismatch])
else:
    print("All rows match.")

All rows match.


In [130]:
mismatch = df_single_label['y_acc'] != df_single_label['y_mag']
# Print mismatched rows and their indices
if mismatch.any():
    print("Rows where 'col1' and 'col2' are different:")
    df_diff['acc_mag_diff'] = df_single_label[mismatch]
    print(df_single_label[mismatch])
else:
    print("All rows match.")

Rows where 'col1' and 'col2' are different:
      y_acc  y_gyr  y_mag
83       25     25      8
1022     25     25      8
1760     25     25      8
2496      7      7      8
2519      8      8      7
2544     38     38      8
2553     38     38      8
2563     38     38      8
2614     38     38      8
2623     38     38      8
2731     38     38      8
3835     25     25      8
4469      8      8      7
4758     25     25      8
6114      8      8     25
6595      8      8      7
6657     38     38      8
6690     38     38      8
6702      8      8     38
6729     38     38      8
6754      8      8     38
6766     38     38      8
6768      8      8     38
6779     38     38      8
6808      8      8     38
8487     25     25      8
9201      7      7      8


In [22]:
mismatch = df_single_label['y_acc'] != df_single_label['y_mic']
# Print mismatched rows and their indices
if mismatch.any():
    print("Rows where 'col1' and 'col2' are different:")
    df_diff['acc_mic_diff'] = df_single_label[mismatch]
    print(df_single_label[mismatch])
else:
    print("All rows match.")

Rows where 'col1' and 'col2' are different:
      y_acc  y_gyr  y_mag  y_mic
19        8      8      2      2
38        3      3      3      8
221      14     14      8      8
455       2      2      2      8
569       8      8      3      3
993       8      8      3      3
1059      3      3      3      8
1070      8      8      8      5
1303      8      8      2      2
1775      8      8      2      2
2470      3      3      3      8
2551      2      2      2      8
2647      4      4      4      8
2688      3      3      3      8
2936      2      2      8      8
3023      8      8      7      7
3233      8      8      8     38
3619      8      8      3      3
3758      5      5      5      8
4256      8      8      3      3
4463      8      8      3      3
4614      8      8      8      2
4665      8      8      2      2
4792      8      8      3      3
5152      8      8      8      2
5266      8      8      4      4
5320      5      5      5      8
5685      8      8      2      2

In [ ]:
index_acc_mag_diff = df_diff['acc_mag_diff'].index.tolist()
index_acc_mic_diff=df_diff['acc_mic_diff'].index.tolist()

In [ ]:
combined_diff = index_acc_mag_diff + index_acc_mic_diff

In [133]:
len(combined_diff)

27

In [134]:
idx_to_be_removed = np.unique(combined_diff) #remove these labels from df_single_label and df_combined


In [135]:
idx_to_be_removed.shape

(27,)

In [136]:
df_combined_after_removing = df_combined.drop(index=idx_to_be_removed.tolist())
df_combined_after_removing = df_combined_after_removing.reset_index(drop=True)

In [137]:
df_single_label_after_removing = df_single_label.drop(index = idx_to_be_removed.tolist())
df_single_label_after_removing = df_single_label_after_removing.reset_index(drop=True)

In [138]:
df_combined_after_removing.shape,df_single_label_after_removing.shape

((9428, 6), (9428, 3))

In [ ]:
df_combined_after_removing.drop(['y_acc','y_gyr','y_mag','y_mic'], axis=1, inplace=True)

In [140]:
df_pre_final_combined_data = pd.concat([df_combined_after_removing, df_single_label_after_removing], axis=1)

In [141]:
df_pre_final_combined_data.head()

,X_acc,X_gyr,X_mag,y_acc,y_gyr,y_mag
0,"[[1594225726.872851, 2.634521, -0.119751, 9.50...","[[1594225726.872851, -0.001065264, 0.01384844,...","[[1594225726.886678, 1.168907e-06, 2.308592e-0...",8,8,8
1,"[[1594225727.0683732, 2.6278998148363666, -0.0...","[[1594225727.0683732, -0.000464741938271, 0.01...","[[1594225727.073816, 1.2370223002132196e-06, 2...",8,8,8
2,"[[1594225727.2736714, 2.662067624954272, -0.10...","[[1594225727.2736714, 0.0001770807704802, 0.01...","[[1594225727.2738602, 1.322222334121227e-06, 2...",8,8,8
3,"[[1594225727.469194, 2.6632931830039435, -0.12...","[[1594225727.469194, -0.0012910583948713, 0.01...","[[1594225727.4739044, 4.76491691183792e-06, 2....",8,8,8
4,"[[1594225727.6744926, 2.6273221483269924, -0.1...","[[1594225727.6744926, 0.0026625376581309, 0.01...","[[1594225727.6739483, 1.1122596115568694e-06, ...",8,8,8


In [ ]:
df_final = df_pre_final_combined_data.drop(['y_gyr','y_mag','y_mic'], axis=1)

In [143]:
df_final.head()

,X_acc,X_gyr,X_mag,y_acc
0,"[[1594225726.872851, 2.634521, -0.119751, 9.50...","[[1594225726.872851, -0.001065264, 0.01384844,...","[[1594225726.886678, 1.168907e-06, 2.308592e-0...",8
1,"[[1594225727.0683732, 2.6278998148363666, -0.0...","[[1594225727.0683732, -0.000464741938271, 0.01...","[[1594225727.073816, 1.2370223002132196e-06, 2...",8
2,"[[1594225727.2736714, 2.662067624954272, -0.10...","[[1594225727.2736714, 0.0001770807704802, 0.01...","[[1594225727.2738602, 1.322222334121227e-06, 2...",8
3,"[[1594225727.469194, 2.6632931830039435, -0.12...","[[1594225727.469194, -0.0012910583948713, 0.01...","[[1594225727.4739044, 4.76491691183792e-06, 2....",8
4,"[[1594225727.6744926, 2.6273221483269924, -0.1...","[[1594225727.6744926, 0.0026625376581309, 0.01...","[[1594225727.6739483, 1.1122596115568694e-06, ...",8


In [ ]:
df_final
# # can  skip if loading from a saved file, as time steps are already removed in the saved file
def strip_timestamp(Xt):
    return np.array([x[:, 1:] for x in Xt])  # keep only sensor axes

X_acc = strip_timestamp(df_final['X_acc'])
X_gyr = strip_timestamp(df_final['X_gyr'])
X_mag = strip_timestamp(df_final['X_mag'])
X_mic = strip_timestamp(df_final['X_mic'])

df_final['X_acc'] = [x for x in X_acc]
df_final['X_gyr'] = [x for x in X_gyr]
df_final['X_mag'] = [x for x in X_mag]
df_final['X_mic'] = [x for x in X_mic]

In [151]:
df_final.head()

,X_acc,X_gyr,X_mag,y_acc
0,"[[2.634521, -0.119751, 9.508227], [2.658472, -...","[[-0.001065264, 0.01384844, -0.006391587], [0....","[[1.168907e-06, 2.308592e-05, -7.498539e-05], ...",8
1,"[[2.6278998148363666, -0.0999802918746799, 9.4...","[[-0.000464741938271, 0.0137126547490805, -0.0...","[[1.2370223002132196e-06, 2.251274205828003e-0...",8
2,"[[2.662067624954272, -0.1033777000414604, 9.47...","[[0.0001770807704802, 0.0131373317071921, -0.0...","[[1.322222334121227e-06, 2.4667943733767385e-0...",8
3,"[[2.6632931830039435, -0.123874540427468, 9.46...","[[-0.0012910583948713, 0.0118730628135427, -0....","[[4.76491691183792e-06, 2.542096956921588e-05,...",8
4,"[[2.6273221483269924, -0.1397141727538777, 9.4...","[[0.0026625376581309, 0.0147363645019998, -0.0...","[[1.1122596115568694e-06, 2.246099611556869e-0...",8


In [ ]:
df_final.to_pickle('./pneumatic_screwdriver_dataset_final.pkl')

## Below implementation is for one hot encoding, if require then implement otherwise ignore it

In [146]:
y_acc = np.array(df_pre_final_combined_data['y_acc'])
unique_classes_acc = np.unique(y_acc)
class_to_index = {label: i for i, label in enumerate(unique_classes_acc)}
y_acc_mapped = np.array([class_to_index[label] for label in y_acc])

y_gyr = np.array(df_pre_final_combined_data['y_gyr'])
unique_classes_gyr = np.unique(y_gyr)
class_to_index = {label: i for i, label in enumerate(unique_classes_gyr)}
y_gyr_mapped = np.array([class_to_index[label] for label in y_gyr])

y_mag = np.array(df_pre_final_combined_data['y_mag'])
unique_classes_mag = np.unique(y_mag)
class_to_index = {label: i for i, label in enumerate(unique_classes_mag)}
y_mag_mapped = np.array([class_to_index[label] for label in y_mag])

# y_mic = np.array(df_pre_final_combined_data['y_mic'])
# unique_classes_mic = np.unique(y_mic)
# class_to_index = {label: i for i, label in enumerate(unique_classes_mic)}
# y_mic_mapped = np.array([class_to_index[label] for label in y_mic])

In [ ]:
# class_to_index,
# mytool = "electric_screwdriver"
'''{14: 'tightening_double',
 7: 'shaking',
 8: 'undefined',
 2: 'tightening',
 5: 'motor_activity_ccw',
 4: 'motor_activity_cw',
 3: 'untightening',
 6: 'manual_motor_rotation'}'''

# "pneumatic_screwdriver"
'''
{7: 'shaking',
 5: 'motor_activity_ccw',
 3: 'untightening',
 38: 'impact',
 8: 'undefined',
 2: 'tightening',
 4: 'motor_activity_cw',
 14: 'tightening_double'}
 '''

# "pneumatic_rivet_gun"
'''
{38: 'impact',
 30: 'pull_trigger_air_with_rivet',
 28: 'pull_trigger_not_removed',
 8: 'undefined',
 25: 'pull_trigger',
 29: 'pull_trigger_air',
 7: 'shaking'}'''

print("class and counts for ACC, GYR, MAG, MIC")
np.unique(y_acc, return_counts=True), np.unique(y_gyr, return_counts=True), np.unique(y_mag, return_counts=True), np.unique(y_mic, return_counts=True)

class and counts for ACC, GYR, MAG, MIC


((array([ 7,  8, 25, 38]), array([ 110, 9273,    5,   40])),
 (array([ 7,  8, 25, 38]), array([ 110, 9273,    5,   40])),
 (array([ 7,  8, 25, 38]), array([ 110, 9273,    5,   40])))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
y_acc = encoder.fit_transform(np.array(y_acc_mapped).reshape(-1, 1))
y_gyr = encoder.fit_transform(np.array(y_gyr_mapped).reshape(-1, 1))
y_mag = encoder.fit_transform(np.array(y_mag_mapped).reshape(-1, 1))
y_mic = encoder.fit_transform(np.array(y_mic_mapped).reshape(-1, 1))

In [ ]:
data_final_X = df_pre_final_combined_data.iloc[:,:4]

In [157]:
data_final_X

,X_acc,X_gyr,X_mag
0,"[[1594225726.872851, 2.634521, -0.119751, 9.50...","[[1594225726.872851, -0.001065264, 0.01384844,...","[[1594225726.886678, 1.168907e-06, 2.308592e-0..."
1,"[[1594225727.0683732, 2.6278998148363666, -0.0...","[[1594225727.0683732, -0.000464741938271, 0.01...","[[1594225727.073816, 1.2370223002132196e-06, 2..."
2,"[[1594225727.2736714, 2.662067624954272, -0.10...","[[1594225727.2736714, 0.0001770807704802, 0.01...","[[1594225727.2738602, 1.322222334121227e-06, 2..."
3,"[[1594225727.469194, 2.6632931830039435, -0.12...","[[1594225727.469194, -0.0012910583948713, 0.01...","[[1594225727.4739044, 4.76491691183792e-06, 2...."
4,"[[1594225727.6744926, 2.6273221483269924, -0.1...","[[1594225727.6744926, 0.0026625376581309, 0.01...","[[1594225727.6739483, 1.1122596115568694e-06, ..."
...,...,...,...
9423,"[[1594228511.3105488, 4.690029969944956, 0.851...","[[1594228511.3105488, -0.7266768781528571, 0.3...","[[1594228511.313104, 1.180391437813212e-05, 1...."
9424,"[[1594228511.5158472, 2.9466941546678376, -0.8...","[[1594228511.5158472, 0.0283724972461223, 0.00...","[[1594228511.5132442, 6.866077159438862e-06, 9..."
9425,"[[1594228511.7113695, 1.0657803993292472, -0.7...","[[1594228511.7113695, -0.1279371339285301, 0.2...","[[1594228511.7133844, 7.882172934647803e-06, 8..."
9426,"[[1594228511.916668, 2.489991690022681, -0.817...","[[1594228511.916668, -0.0265452539887325, 0.03...","[[1594228511.9135246, 8.044007390705935e-06, 9..."


In [158]:
y = y_acc

In [159]:
y.shape

(9428, 4)

In [160]:
data_final_y = pd.DataFrame({'y':list(y)})
data_final_y

,y
0,"[0.0, 1.0, 0.0, 0.0]"
1,"[0.0, 1.0, 0.0, 0.0]"
2,"[0.0, 1.0, 0.0, 0.0]"
3,"[0.0, 1.0, 0.0, 0.0]"
4,"[0.0, 1.0, 0.0, 0.0]"
...,...
9423,"[0.0, 1.0, 0.0, 0.0]"
9424,"[0.0, 1.0, 0.0, 0.0]"
9425,"[0.0, 1.0, 0.0, 0.0]"
9426,"[0.0, 1.0, 0.0, 0.0]"


In [161]:
final_saved_data = pd.concat([data_final_X, data_final_y],axis=1)

In [164]:
final_saved_data

,X_acc,X_gyr,X_mag,y
0,"[[2.634521, -0.119751, 9.508227], [2.658472, -...","[[-0.001065264, 0.01384844, -0.006391587], [0....","[[1.168907e-06, 2.308592e-05, -7.498539e-05], ...","[0.0, 1.0, 0.0, 0.0]"
1,"[[2.6278998148363666, -0.0999802918746799, 9.4...","[[-0.000464741938271, 0.0137126547490805, -0.0...","[[1.2370223002132196e-06, 2.251274205828003e-0...","[0.0, 1.0, 0.0, 0.0]"
2,"[[2.662067624954272, -0.1033777000414604, 9.47...","[[0.0001770807704802, 0.0131373317071921, -0.0...","[[1.322222334121227e-06, 2.4667943733767385e-0...","[0.0, 1.0, 0.0, 0.0]"
3,"[[2.6632931830039435, -0.123874540427468, 9.46...","[[-0.0012910583948713, 0.0118730628135427, -0....","[[4.76491691183792e-06, 2.542096956921588e-05,...","[0.0, 1.0, 0.0, 0.0]"
4,"[[2.6273221483269924, -0.1397141727538777, 9.4...","[[0.0026625376581309, 0.0147363645019998, -0.0...","[[1.1122596115568694e-06, 2.246099611556869e-0...","[0.0, 1.0, 0.0, 0.0]"
...,...,...,...,...
9423,"[[4.690029969944956, 0.8514012726289639, 9.941...","[[-0.7266768781528571, 0.3138531572994301, -0....","[[1.180391437813212e-05, 1.1854180840546695e-0...","[0.0, 1.0, 0.0, 0.0]"
9424,"[[2.9466941546678376, -0.8022670686811043, 7.8...","[[0.0283724972461223, 0.0094422577216369, -0.1...","[[6.866077159438862e-06, 9.06951917731166e-06,...","[0.0, 1.0, 0.0, 0.0]"
9425,"[[1.0657803993292472, -0.7472604628032844, 9.4...","[[-0.1279371339285301, 0.2382438976269226, 0.1...","[[7.882172934647803e-06, 8.75417901633805e-06,...","[0.0, 1.0, 0.0, 0.0]"
9426,"[[2.489991690022681, -0.8179056711338195, 9.33...","[[-0.0265452539887325, 0.0386085921959368, 0.0...","[[8.044007390705935e-06, 9.60648869971881e-06,...","[0.0, 1.0, 0.0, 0.0]"


In [ ]:
# # can  skip if loading from a saved file, as time steps are already removed in the saved file
def strip_timestamp(Xt):
    return np.array([x[:, 1:] for x in Xt])  # keep only sensor axes

X_acc = strip_timestamp(final_saved_data['X_acc'])
X_gyr = strip_timestamp(final_saved_data['X_gyr'])
X_mag = strip_timestamp(final_saved_data['X_mag'])
X_mic = strip_timestamp(final_saved_data['X_mic'])
final_saved_data['X_acc'] = [x for x in X_acc]
final_saved_data['X_gyr'] = [x for x in X_gyr]
final_saved_data['X_mag'] = [x for x in X_mag]
final_saved_data['X_mic'] = [x for x in X_mic]

In [ ]:
# os.chdir(r"E:/MS_AI/Sem-IV/Seminar-ADLTS/exAI-timeseries-tool-tracking-data/.cleaned_data")
# final_saved_data.to_pickle('./electric_screwdriver_dataset_final.pkl')
final_saved_data.to_pickle('./pneumatic_screwdriver_dataset_final.pkl')
# final_saved_data.to_pickle('./pneumatic_rivet_gun_dataset_final.pkl')

## Load data from saved file to cut down above implementations

In [169]:

df = pd.read_pickle(r'E:/MS_AI/Sem-IV/Seminar-ADLTS/exAI-timeseries-tool-tracking-data/.cleaned_data/pneumatic_rivet_gun_dataset_final.pkl')

In [170]:
df['X_acc'][0]

array([[ 2.634521  , -0.119751  ,  9.508227  ],
       [ 2.658472  , -0.1437012 ,  9.479487  ],
       [ 2.6341212 , -0.09115959,  9.48562559],
       [ 2.66790364, -0.05228997,  9.48961569],
       [ 2.6556065 , -0.11684295,  9.46752262],
       [ 2.632099  , -0.10727523,  9.47806626],
       [ 2.62544437, -0.13538746,  9.47520037],
       [ 2.66349098, -0.09179519,  9.4618473 ],
       [ 2.65611331, -0.09815948,  9.46430656],
       [ 2.65759982, -0.10842651,  9.46119918],
       [ 2.6622065 , -0.1101709 ,  9.452858  ],
       [ 2.67431032, -0.09175688,  9.45811255],
       [ 2.64932191, -0.0862207 ,  9.49571461],
       [ 2.6951173 , -0.08970856,  9.52217905],
       [ 2.69268834, -0.08768733,  9.45094956],
       [ 2.67917119, -0.07971904,  9.45297588],
       [ 2.67119226, -0.05410136,  9.48922688],
       [ 2.69369088, -0.08156897,  9.48582789],
       [ 2.69745676, -0.11842147,  9.46462826],
       [ 2.66371036, -0.11518516,  9.48382864],
       [ 2.62789981, -0.09998029,  9.498

## Require step to make data samples in shape of (num_samples, window_frames, features)

In [100]:
import torch
all_acc = []
for windows in df['X_acc']:  # windows is a list/array of (time_steps, 3)
        all_acc.append(windows)  # append the full window, shape (time_steps, 3)
X_acc = torch.tensor(all_acc, dtype=torch.float32)  # shape: (total_windows, time_steps, 3)

all_gyr = []
for windows in df['X_gyr']:
    all_gyr.append(windows)
X_gyr = torch.tensor(all_gyr, dtype=torch.float32)  # shape: [7187, 41, 3]

all_mag = []
for windows in df['X_mag']:
    all_mag.append(windows)
X_mag = torch.tensor(all_mag, dtype=torch.float32)  # shape: [7187, 61, 3]

all_mic = []
for windows in df['X_mic']:
    all_mic.append(windows)
X_mic = torch.tensor(all_mic, dtype=torch.float32)  # shape: [7187, 3200, 1]


C:\Users\Akash\AppData\Local\Temp\ipykernel_29056\857492976.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X_acc = torch.tensor(all_acc, dtype=torch.float32)  # shape: (total_windows, time_steps, 3)


In [67]:
# verify the output type and shape eg. torch.size([7187, 41, 3])
X_gyr.shape, X_acc.shape, X_mag.shape, X_mic.shape

(torch.Size([7187, 41, 3]),
 torch.Size([7187, 41, 3]),
 torch.Size([7187, 62, 3]),
 torch.Size([7187, 3200, 1]))

In [101]:
X_gyr.shape, X_acc.shape, X_mag.shape, X_mic.shape

(torch.Size([6137, 41, 3]),
 torch.Size([6137, 41, 3]),
 torch.Size([6137, 62, 3]),
 torch.Size([6137, 3200, 1]))